### Without Pipeline

In [37]:
import numpy as np
import pandas as pd

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [39]:
df = pd.read_csv(r"F:\Education\100 Days of ML\Feature Engineering\titanic.csv")

df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
751,752,1,3,"Moor, Master. Meier",male,6.0,0,1,392096,12.4750,E121,S
496,497,1,1,"Eustis, Miss. Elizabeth Mussey",female,54.0,1,0,36947,78.2667,D20,C
691,692,1,3,"Karun, Miss. Manca",female,4.0,0,1,349256,13.4167,NaN,C
509,510,1,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,NaN,S
520,521,1,1,"Perreault, Miss. Anne",female,30.0,0,0,12749,93.5000,B73,S


In [40]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [41]:
df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
858,1,3,female,24.0,0,3,19.2583,C
502,0,3,female,NaN,0,0,7.6292,Q
766,0,1,male,NaN,0,0,39.6000,C
308,0,2,male,30.0,1,0,24.0000,C
47,1,3,female,NaN,0,0,7.7500,Q


In [42]:
X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [44]:
y_train.head(2)

331    0
733    0
Name: Survived, dtype: int64

In [45]:
df.isnull().sum().sort_values(ascending=False).head(3)

Age         177
Embarked      2
Survived      0
dtype: int64

In [46]:
si_age = SimpleImputer() # mean
si_embarked = SimpleImputer(strategy='most_frequent') # mode

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']]) 

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [47]:
# X_test_embarked

In [48]:
ohe_sex = OneHotEncoder(sparse_output =False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

In [49]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [52]:
X_train_rem = X_train.drop(['Sex', 'Embarked', 'Age'], axis=1)

X_test_rem = X_test.drop(['Sex', 'Embarked', 'Age'], axis=1)

In [55]:
X_trained_transformed = np.concatenate((X_train_rem, X_train_age, X_train_sex, X_train_embarked), axis=1)
X_test_transformed = np.concatenate((X_test_rem, X_test_age, X_test_sex, X_test_embarked), axis=1)

In [59]:
X_trained_transformed.shape
X_test_transformed.shape

(179, 10)

In [61]:
model = DecisionTreeClassifier()

model.fit(X_trained_transformed, y_train)

DecisionTreeClassifier()

In [63]:
y_pred = model.predict(X_test_transformed)

y_pred

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1], dtype=int64)

In [64]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7932960893854749

### With Pipeline

In [65]:
import numpy as np
import pandas as pd

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [68]:
titanic = pd.read_csv('titanic.csv')

titanic.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
109,110,1,3,"Moran, Miss. Bertha",female,NaN,1,0,371110,24.1500,NaN,Q
91,92,0,3,"Andreasson, Mr. Paul Edvin",male,20.0,0,0,347466,7.8542,NaN,S
64,65,0,1,"Stewart, Mr. Albert A",male,NaN,0,0,PC 17605,27.7208,NaN,C
529,530,0,2,"Hocking, Mr. Richard George",male,23.0,2,1,29104,11.5000,NaN,S
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S


In [78]:
titanic.isnull().sum().sort_values(ascending=False)

Age         177
Embarked      2
Survived      0
Pclass        0
Sex           0
SibSp         0
Parch         0
Fare          0
dtype: int64

In [69]:
titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [70]:
titanic.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
117,0,2,male,29.0,1,0,21.000,S
95,0,3,male,NaN,0,0,8.050,S
729,0,3,female,25.0,1,0,7.925,S
385,0,2,male,18.0,0,0,73.500,S
880,1,2,female,25.0,0,1,26.000,S


In [71]:
X = titanic.drop("Survived", axis=1)
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [72]:
y = titanic['Survived']

y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [79]:
# Imputation Transfer

trf1 = ColumnTransformer( [
    ('impute_age', SimpleImputer(), [2]), # 2 is index value of age
    ('impute_embarked', SimpleImputer(strategy='most_frequent'),[6]) # 6 is index value of embarked
    ], remainder='passthrough') # passthrough make sure to include remaining columns

In [80]:
trf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [84]:
# one hot encoding

trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore',drop=None), [1,6]) # encoding both sex and embarked at once
], remainder='passthrough')

# Note: Here we are not using drop first in OHE because we are using Decision Tree but for
# Linear Data we also include drop firt like drop='first'

In [85]:
trf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 [1, 6])])

In [86]:
# scaling

trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

 # scaling all the columns. 10 columns becasue after OneHotEncoder our dimensions will increas

In [87]:
trf3

ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))])

In [88]:
# feature selection

trf4 = SelectKBest(score_func=chi2, k=7)

In [89]:
trf4

SelectKBest(k=7, score_func=<function chi2 at 0x000001503ABB15E0>)

In [90]:
# train the model

trf5 = DecisionTreeClassifier()

In [91]:
trf5

DecisionTreeClassifier()

**Create Pipeline**

In [93]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5),
])

In [94]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=7,
                             score_func=<function chi2 at 0x000001503ABB15E0>)),
                ('trf5', DecisionTreeClassifier())])

**Exploring Pipelines**

In [95]:
pipe.named_steps   # we can look at all the steps of the pipelines

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=7, score_func=<function chi2 at 0x000001503ABB15E0>),
 'trf5': DecisionTreeClassifier()}

In [101]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_  # like this we can go deep into the pipelines

array([29.49884615])

In [113]:
pipe.named_steps['trf2'].transformers_[0][1].categories_[0]

array(['C', 'Q', 'S'], dtype=object)

In [115]:
y_pred = pipe.predict(X_test)

y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [116]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.6256983240223464

**Cross Validation using pipelines**

In [118]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

What if we want to use the Pipelines in Production?

**Exporting the Pipeline**

In [119]:
import pickle

pickle.dump(pipe, open('pipe.pkl', 'wb'))

The advantage of using Pipeline is, You don't need to repeat the same preprocessing steps in production envirnment even after getting new data or making changes in the training data. 